In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

import pinecone
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ctransformers
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Pinecone
from ctransformers import AutoModelForCausalLM

c:\Users\diyak\anaconda3\envs\mchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()  # loads from .env by default
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_HOST = os.getenv("PINECONE_HOST")
HUGGINGFACEHUB_API_TOKEN=os.getenv('HUGGINGFACEHUB_API_TOKEN')


In [3]:

def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents


In [4]:
extracted_pdf=load_pdf("data/")

In [5]:
# extracted_pdf

In [6]:
def text_split(extracted_data):
  splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
  text_chunks=splitter.split_documents(extracted_data)
  return text_chunks


In [7]:
chunks=text_split(extracted_pdf)
len(chunks)

5859

In [8]:
len(chunks[39].page_content)

487

In [9]:

def download_huggingface_embeddings():
  
  model_ = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  
  print(model)


In [10]:
model_ = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print(embedding)

model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


In [12]:
len(model_.encode('how are you'))

384

In [18]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [27]:
if 'medicalchatbot' not in pc.list_indexes().names():
        pc.create_index(
            name='medicalchatbot',
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )


In [28]:
# import os
# from pinecone import Pinecone, ServerlessSpec
# from langchain_pinecone import PineconeVectorStore

# # Init Pinecone
# pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# # Define your index name
# index_name = "medicalchatbot"

# # Create vector store from documents
# vector_store = PineconeVectorStore.from_documents(
#     documents=chunks,
#     embedding=embedding,
#     index_name=index_name
# )


In [31]:
index_name='medicalchatbot'
index = pc.Index(index_name)
vectorstore = PineconeVectorStore(index=index, embedding=embedding)

In [32]:
template="""
use the following peice of information to answer the user's question. If you do not know the answer just simply say that you do not know the answer , do not try to make up an answer.
Context:{context}
Question={question}
Only give the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=template,input_variables=["context","question"])

In [34]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
    model_path_or_repo_id="Model/llama-2-7b-chat.Q3_K_S.gguf",  # path to your .gguf file
    model_type="llama",  # required
     
    max_new_tokens= 512,
    temperature= 0.8
    
)


In [35]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.chains.question_answering import load_qa_chain

In [36]:
from langchain_community.llms import CTransformers
llm = CTransformers(
    model="Model/llama-2-7b-chat.Q3_K_S.gguf",
    model_type="llama",  # or "mistral", "gpt2", etc.
    config={"max_new_tokens": 20, "temperature": 0.7}
)

In [37]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.chains.question_answering import load_qa_chain
def retrieve_query(query,k=2):
  matching_results=vectorstore.similarity_search(query,k=k)
  return matching_results

from langchain.chains.combine_documents import create_stuff_documents_chain
chain = create_stuff_documents_chain(llm=llm, prompt=PROMPT)
def retrieve_answers(query):
    doc_search = retrieve_query(query)  # retrieves relevant documents
    response = chain.invoke({"context": doc_search, "question": query})

    return response
our_query = "what is hernia?"
try:
  answer = retrieve_answers(our_query)
  print(answer)

except Exception as e:
    print("ERROR:", e)


Hernia is a condition where an organ or tissue pushes through a weakened area in


In [67]:
while True:
  user_input=input('Input prompt:')
  result=retrieve_answers(user_input)
  print('Response:',result)

Response: Hera is not associated with any specific body tissue or function, so I cannot provide information on


KeyboardInterrupt: 